In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re


In [43]:
propertytype = {}
propertytype[0] = 0
propertytype["appartment"] = 1
propertytype["apartment"] = 1
propertytype["departamento"] = 1
propertytype["house"] = 2
propertytype["casa"] = 2
propertytype["PH"] = 3
propertytype["ph"] = 3
propertytype["store"] = 4
propertytype["local"] = 4

def date_to_float(dt64):
    return (dt64 - np.datetime64('2013-01-01T00:00:00Z')) / np.timedelta64(1, 's')


def define_category(d):
    if("picina" in d or "garage" in d):
        return 25

    if("pileta" in d or "cochera" in d):
        return 20

    if("gimnasio" in d):
        return 15
    
    if("sum" in d):
        return 10
    
    if("reciclar" in d or "refaccionar" in d ):
        return -20
    return 0
   

def fill_nan_and_convert_to_float(datos):
    columns = ['created_on','property_type','lat','lon', 'place_name','state_name',
                 'surface_total_in_m2','surface_covered_in_m2','description',
                 'floor','rooms','expenses', 'price_aprox_usd','price_usd_per_m2']
   
    datos = datos.loc[:,columns]
    
    #para el tamaño total de la propiedad uso el promedio
    datos["surface_total_in_m2"] = datos['surface_total_in_m2'].fillna(datos['surface_total_in_m2'].mean())
    #tamaño cubierto, tomo el total de la fila
    datos["surface_covered_in_m2"] = datos['surface_covered_in_m2'].fillna(datos['surface_total_in_m2'])
  
    
    #relleno el piso y las habitaciones con la moda
    datos["floor"] = datos['floor'].fillna(datos.dropna(subset=['floor']).floor.value_counts().idxmax())
    datos["rooms"] = datos['rooms'].fillna(datos.dropna(subset=['rooms']).rooms.value_counts().idxmax())
    
    #tipo de propiedad
    datos["property_type"] = datos.property_type.map(lambda t: propertytype.get(t))
    
    #descripcion
    datos["description"] = datos.description.map(lambda d: define_category(str(d)))
    
    
    #fecha
    datos["created_on"] =  pd.to_datetime(datos['created_on'])
    datos['created_on'] = datos['created_on'].map(lambda dt64 :date_to_float(dt64))
    
    #barrios a numeros
    datos["place_name"] = pd.Categorical(datos.place_name)
    datos["place_name"] = datos.place_name.cat.codes
    
    #zonas a numeros
    datos["state_name"] = pd.Categorical(datos.state_name)
    datos["state_name"] = datos.state_name.cat.codes
    
    #latitud y longitud la relleno con el promedio del barrio
    datos['lat'] = datos.groupby('place_name')['lat'].apply(lambda x: x.fillna(x.mean()))
    datos['lon'] = datos.groupby('place_name')['lon'].apply(lambda x: x.fillna(x.mean()))
    
    
     #si en el campo de las expensas dice no, entonces le asigno expensas cero
    datos["expenses"] = datos["expenses"].map(lambda exp: "0" if(re.search(str(exp), 'no', re.IGNORECASE)) else str(exp))
                                                            
    #si no tiene un valor numerico lo cambio a nan para luego asignarle el promedio del barrio                                                   
    datos["expenses"] = datos["expenses"].map(lambda exp: re.sub("[^0-9]", "",str(exp) ))
    datos["expenses"] = datos["expenses"].map(lambda exp: np.NaN if(str(exp)== "") else exp)
    datos["expenses"] = datos["expenses"].map(lambda exp: float(exp))
    datos["expenses"] = datos.groupby('place_name')['expenses'].apply(lambda x: x.fillna(x.mean()))
    
    #precios
    datos['price_aprox_usd'] = datos['price_aprox_usd'].map(lambda p: float(p))
    datos['price_usd_per_m2'] = datos['price_usd_per_m2'].map(lambda p: float(p))
   
    #por si queda algun nan
    datos.fillna(0, inplace=True)
    return datos

In [3]:
def delete_signs(text):
    return ''.join([i if ord(i) < 128 else ' ' for i in text])
def delete_accent_mark(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def unificate_description(d):
    if(pd.notnull(d)):
        return delete_signs(delete_accent_mark(d).lower())
    return d
    

In [4]:
#lista con todos los barrios posibles sin repetir (sin tildes y en minuscula)
def get_neighborhoods(properati_df,nombre_conj_barios):
    neighborhoods = properati_df[pd.notnull(properati_df['place_name'])].place_name
    neighborhoods = neighborhoods.drop_duplicates(keep='first')
    neighborhoods = neighborhoods.map(lambda b: delete_accent_mark(b).lower())
    neighborhoods.replace(nombre_conj_barios, "sin barrio", inplace = True)
    return neighborhoods



def get_place_name(data):
    places =[]
    for index, row in data.iterrows():
        if(pd.notnull(row['place_name_y']) and (row['place_name_y']!="sin barrio") ):
            places.append(row['place_name_y'].title())
        else:
            if(pd.notnull(row['place_name_x'])):
                places.append(row['place_name_x'].title())
            else:
                places.append(row['place_name_x'])
    return places



def assign_neighborhoods(description,neighborhoods):
    for neighborhood in neighborhoods:
        if(str(description.encode('utf-8')).find(neighborhood) >= 0):
              return neighborhood
    return "sin barrio"




def guess_neighborhoods(df,  nombre_conj_barios):
    
    neighborhoods = get_neighborhoods(df,nombre_conj_barios)

    sin_barrio = df.loc[df.place_name.str.contains(nombre_conj_barios , na=False),:]
    sin_barrio.place_name = sin_barrio.description.map(lambda description: assign_neighborhoods(description,neighborhoods))

    
    barrio_asignado = sin_barrio.loc[sin_barrio.place_name.str.contains("sin barrio", na=False) == False,:]
    
    #columnas_no_price = ['id', 'created_on', 'property_type', 'operation',
    #   'place_with_parent_names', 'country_name', 'state_name',
    #   'lat-lon', 'lat', u'lon', 'surface_total_in_m2',
    #   'surface_covered_in_m2', 'floor', 'rooms', 'expenses',
    #   'description']
    
    columnas_no_price =['country_name','created_on','currency','description',
       'expenses','extra','floor', 'geonames_id','id',
       'image_thumbnail', 'lat', 'lat-lon', 'lon', 'operation',
        'place_with_parent_names', 'price',
       'price_aprox_local_currency', 'price_aprox_usd', 'price_per_m2',
       'price_usd_per_m2', 'properati_url', 'property_type', 'rooms',
       'state_name', 'surface_covered_in_m2', 'surface_in_m2',
       'surface_total_in_m2','title']
    
    
    properati_barrios = pd.merge(df, barrio_asignado,how='outer', on=columnas_no_price)
    
    properati_barrios['place_name']= get_place_name(properati_barrios)
    columnas_no_price.append('place_name')
    properati_barrios=properati_barrios.loc[:,columnas_no_price]
    
    return properati_barrios

In [48]:
def get_df_properati_to_predict(df, file_name):

    df.description = df.description.map(lambda d: unificate_description(d))
    
    guess_neighborhoods(df,"Capital Federal")
    guess_neighborhoods(df,"Buenos Aires Interior")
    
    return fill_nan_and_convert_to_float(df)


In [49]:
properati_filtered= pd.read_csv('../tp1/properati_filtered1.csv',encoding='UTF-8')
datos = get_df_properati_to_predict(properati_filtered, 'nada')


/home/jaz/fiuba/orga-de-datos-7506/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  


In [50]:
datos


,created_on,property_type,lat,lon,place_name,state_name,surface_total_in_m2,surface_covered_in_m2,description,floor,rooms,expenses,price_aprox_usd,price_usd_per_m2
0,94953600.0,2,-35.050505,-59.710847,156,3,1132.0,251.0,20,1.0,3.0,23697.142857,440000.0,388.692580
1,94953600.0,1,-34.603714,-58.381581,430,3,52.0,52.0,20,1.0,1.0,10.000000,125000.0,2403.846154
2,94953600.0,4,-34.628492,-58.390903,179,3,219.0,190.0,0,1.0,3.0,45000.000000,240000.0,1095.890411
3,95040000.0,3,-34.614815,-58.371848,431,3,441.0,356.0,0,1.0,7.0,20271.453488,690000.0,1564.625850
4,95040000.0,1,-34.594032,-58.381577,102,3,180.0,173.0,0,1.0,4.0,40409.760766,360000.0,2000.000000
5,95126400.0,3,-34.627522,-58.358761,144,3,120.0,60.0,0,2.0,3.0,8900.000000,115000.0,958.333333
6,95126400.0,1,-34.550781,-58.454624,360,3,56.0,56.0,0,1.0,3.0,32000.000000,147000.0,2625.000000
7,95126400.0,1,-34.550781,-58.454624,133,3,56.0,56.0,0,1.0,3.0,32000.000000,147000.0,2625.000000
8,95126400.0,1,-34.615605,-58.366759,390,3,47.0,42.0,25,1.0,1.0,71002.000000,185000.0,3936.170213
9,95126400.0,4,-34.623500,-58.486231,236,3,30.0,25.0,0,1.0,3.0,15410.674157,56000.0,1866.666667


In [ ]:

data.to_csv('data_filled_ready_to_train.csv',encoding='utf-8',index=False)